In [ ]:
import os
import datasets

In [ ]:
data = datasets.load_dataset('./loadingScript_imda_part3.py', "Audio Same CloseMic", split="train", streaming=True)

print(next(iter(data)))

In [ ]:
from IPython.display import Audio, display

index = 0
for i in data:
    if index == 1319: 
        
        display_data = i['audio']

        print(f'sampling rate: {display_data["sampling_rate"]}')
        display(Audio(display_data["array"], rate=display_data["sampling_rate"]))
        print(f'transcript: {i["transcript"]}')
        # print(f'gender: {i["gender"]}')
        # print(f'race: {i["race"]}')
        print(f'interval: {i["interval"]}')
        break
    index+=1

In [ ]:
import glob
path_to_data = 'D:\IMDA - National Speech Corpus\PART1'
mic = "CHANNEL0"
speaker = '0001'
metadata_path = os.path.join(path_to_data, "DATA", mic, "SCRIPT", mic[-1]+speaker+'*.TXT')
print(metadata_path)
script_list = glob.glob(metadata_path)
print(script_list)
d = {}
for script in script_list:
    line_num = 0
    with open(script, encoding='utf-8-sig') as f:
        for line in f:
            if line_num == 0:
                key = line.split("\t")[0]
                line_num += 1

            elif line_num == 1:
                d[key] = line.strip()
                line_num -= 1

    break

print(d)



In [ ]:
archive_path = r'D:\IMDA - National Speech Corpus\PART1\DATA\CHANNEL0\WAVE\SPEAKER0001.zip'
from datasets.download import DownloadManager
dl_manager = DownloadManager()
if os.path.exists(archive_path):
    audio_files = dl_manager.iter_archive(archive_path)
    for path, f in audio_files:
        print(path)
        break
else:
    print('error')

### Exploring speaker information for train-test split

In [ ]:
import pandas as pd

p1_speaker_df = pd.read_excel("./Speaker Information (Part 1).XLSX", dtype={'SCD/PART1': object})
p2_speaker_df = pd.read_excel("./Speaker Information (Part 2).XLSX", dtype={'SCD/PART2': object})
p1_speaker_df.head()

In [ ]:
p1_speaker_df["SEX"].value_counts()

In [ ]:
p1_speaker_df["ACC"].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X = p1_speaker_df[(p1_speaker_df["ACC"]=="CHINESE") & (p1_speaker_df["SEX"]=="F")]
print("number of rows: ", len(X))
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True)

print("number of rows in train: ",len(X_train))
print("number of rows in test: ",len(X_test))

In [ ]:
genderlist = p1_speaker_df["SEX"].unique()
racelist = p1_speaker_df["ACC"].unique()
train_speaker_ids = []
test_speaker_ids = []

for gender in genderlist:
    for race in racelist:
        X = p1_speaker_df[(p1_speaker_df["ACC"]==race) & (p1_speaker_df["SEX"]==gender)]
        X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True)
        train_speaker_ids.extend(X_train["SCD/PART1"])
        test_speaker_ids.extend(X_test["SCD/PART1"])
        

In [ ]:
p1_speaker_df.loc[p1_speaker_df["SCD/PART1"]=="0001"].iloc[0]["SEX"]

### PART 3

In [ ]:
from utils import cleanup_string

line = "<fil> you can go first  you guys are going to stand here <fil>"
print(cleanup_string(line))

In [ ]:
path_to_data = r'C:\Users\calic\Downloads\huggingface-dataset\imda-dataset\IMDA - National Speech Corpus\PART3'
mics = ["Audio Same BoundaryMic", "Audio Same CloseMic", "Audio Separate IVR", "Audio Separate StandingMic"]
for mic in mics:
    for (root,dirs,files) in os.walk(os.path.join(path_to_data, mic), topdown=True):
        print(root)
        # print(dirs)
        # print(files)



In [ ]:
for (root,dirs,files) in os.walk(os.path.join(path_to_data, mics[2]), topdown=True):
    if len(files) != 0:
        for file in files:
            print("path: ", os.path.join(root, file))
            if mics[2].split()[1] == "Same":
                print("script path: ", os.path.join(path_to_data, "Scripts Same", file[:-4]+".TextGrid"))
            elif mics[2].split()[1] == "Separate":
                if mics[2] == "Audio Separate IVR":
                    print("script path: ", os.path.join(path_to_data, "Scripts Separate", os.path.split(root)[-1]+"_"+file[:-4]+".TextGrid"))
                elif mics[2] == "Audio Separate StandingMic":
                    print("script path: ", os.path.join(path_to_data, "Scripts Separate", file[:-4]+".TextGrid"))

In [ ]:
# mics = ["Audio Same BoundaryMic", "Audio Same CloseMic", "Audio Separate IVR", "Audio Separate StandingMic"]
mics=["Audio Same CloseMic"]
audio_list = []
for mic in mics:
    for (root,dirs,files) in os.walk(os.path.join(path_to_data, mic), topdown=True):
        if len(files) != 0:
            for file in files:
                # get audio path
                audio_path = os.path.join(root, file)
                audio_list.append(audio_path)

In [ ]:
from textgrid import textgrid
import soundfile as sf
from clean_transcript import cleanup_string

INTERVAL_MAX_LENGTH = 25

id_=0
for audio_path in audio_list:
    print("AUDIO: ",audio_path)
    file = os.path.split(audio_path)[-1]
    folder = os.path.split(os.path.split(audio_path)[0])[-1]

    # get script_path
    if folder.split("_")[0] == "conf":
        # mic == "Audio Separate IVR"
        script_path = os.path.join(path_to_data, "Scripts Separate", folder+"_"+file[:-4]+".TextGrid")
    elif folder.split()[1] == "Same":
        # mic == "Audio Same CloseMic IVR"
        script_path = os.path.join(path_to_data, "Scripts Same", file[:-4]+".TextGrid")
    elif folder.split()[1] == "Separate":
        # mic == "Audio Separate StandingMic":
        script_path = os.path.join(path_to_data, "Scripts Separate", file[:-4]+".TextGrid")

    tg = textgrid.TextGrid.fromFile(script_path)
    data, sr = sf.read(audio_path)
    result = {}
    i = 0
    intervalLength = 0
    intervalStart = 0
    transcript_list = []
    filepath = os.path.join(path_to_data, 'tmp_clip.wav')
    while i < (len(tg[0])-1):
        transcript = cleanup_string(tg[0][i].mark)
        if intervalLength == 0 and len(transcript) == 0:
            intervalStart = tg[0][i].maxTime
            i+=1
            continue
        intervalLength += tg[0][i].maxTime-tg[0][i].minTime
        if intervalLength > INTERVAL_MAX_LENGTH:
            print(f"INTERVAL LONGER THAN {intervalLength}")
            result["transcript"] = transcript
            result["interval"] = "start:"+str(tg[0][i].minTime)+", end:"+str(tg[0][i].maxTime)
            result["audio"] = {"path": audio_path, "bytes": data[int(tg[0][i].minTime*sr):int(tg[0][i].maxTime*sr)], "sampling_rate":sr}
            print( id_, result)
            id_+= 1
            intervalLength = 0
        else:
            if (intervalLength + tg[0][i+1].maxTime-tg[0][i+1].minTime) < INTERVAL_MAX_LENGTH:
                if len(transcript) != 0:
                    transcript_list.append(transcript)
                i+=1
                continue
            if len(transcript) == 0:
                spliced_audio = data[int(intervalStart*sr):int(tg[0][i].minTime*sr)]
            else:
                transcript_list.append(transcript)
                spliced_audio = data[int(intervalStart*sr):int(tg[0][i].maxTime*sr)]
            sf.write(filepath, spliced_audio, sr)
            result["interval"] = "start:"+str(intervalStart)+", end:"+str(tg[0][i].maxTime)
            result["audio"] = {"path": filepath, "bytes": spliced_audio, "sampling_rate":sr}
            result["transcript"] = ' '.join(transcript_list)
            print(id_, result)
            id_+= 1
            intervalLength=0
            intervalStart=tg[0][i].maxTime
            transcript_list = []
        i+=1